【問題1】ブレンディングのスクラッチ実装

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

In [10]:
df = pd.read_csv("train.csv")
X = df.drop("SalePrice",axis=1)
X = X.loc[:,["GrLivArea", "YearBuilt"]]
y = df.loc[:,"SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   test_size=0.2,
                                                   random_state=0)
df_train = pd.concat([X_train, y_train], axis=1)
df_train.shape
X_test.shape
#df_train = pd.concat([X_train, y_train], axis=1)
#df_train.columns[-1]
y_train.ravel()

array([314813, 109500, 163500, ..., 112000, 234000, 221000])

In [3]:

class blending():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, kf_splits, models):
        self.kf_splits = kf_splits
        self.models = models
        
    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        self.fits = {}
        n=1
        kf = KFold(n_splits=self.kf_splits)
        for train_index, test_index in kf.split(df_train):
            X_train = df_train.iloc[train_index,:].drop([target_name], axis=1)
            y_train = df_train.iloc[train_index,:].loc[:,[target_name]]
            X_test = df_train.iloc[test_index,:].drop([target_name], axis=1)
            y_test = df_train.iloc[test_index,:].loc[:,[target_name]]
            
            for model in self.models.values():
                self.fits[n] = model.fit(X_train, y_train)
                n += 1
    def predict(self, X):
        predicts = np.empty((X.shape[0], 1))
        for fit in self.fits.values():
            predicts = np.concatenate([predicts, fit.predict(X).reshape(X.shape[0], 1)], axis=1)
            
        predicts=predicts[:,1:]
        predicts=np.mean(predicts,axis=1)
        return predicts

In [4]:
def model_comparison(_X_train, _y_train, _X_test, _y_test, models):
    _df_results = pd.DataFrame()
    for model_name, model in models.items():
        model.fit(_X_train, _y_train)
        _df_result = pd.DataFrame({model_name : [mean_squared_error(_y_test, model.predict(_X_test)),
                                                 np.sqrt(mean_squared_error(_y_test, model.predict(_X_test))),
                                                 #r2_score(_y_test, model.predict(_X_test))
                                                ]}, 
                                    index = ["mean_squared_error", "rmse"])
        _df_results = pd.concat([_df_results, _df_result],axis=1)       
    return _df_results

In [14]:
models = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0)
        }
models2 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        }
models3 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        "RandomForest":RandomForestRegressor(random_state=0)
        }

blend_3split = blending(2, models3)
blend_3split.fit(X_train, y_train)

#blend_3split.predict(X_test)

/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver

In [15]:
models2 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        }
models3 = {
        "LinearRegression" : LinearRegression(),
        "SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0),
        "LogisticRegression":LogisticRegression(random_state=0),
        "RandomForest":RandomForestRegressor(random_state=0)
        }
models_test = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"LogisticRegression":LogisticRegression(random_state=0),
        #"DecisionTree":DecisionTreeRegressor(random_state=0),
        "Blending1":blending(2, models),
        "Blending2":blending(2, models2),
        "Blending3":blending(2, models3)
        }

model_comparison(X_train, y_train, X_test, y_test, models_test)

/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Users/shunnadamoto/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning

,LinearRegression,Blending1,Blending2,Blending3
mean_squared_error,2.942067e+09,3.294162e+09,3.421466e+09,3.029193e+09
rmse,5.424082e+04,5.739479e+04,5.849330e+04,5.503811e+04


【問題2】バギングのスクラッチ実装

In [16]:

class bagging():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, n_iter, models, test_size=0.2):
        self.n_iter = n_iter
        self.models = models
        self.test_size = test_size
        
    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        self.fits = {}
        col=1
        for n in range(self.n_iter):
            X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,
                                                               test_size=self.test_size,
                                                               shuffle = True)
            for model in models.values():
                self.fits[n] = model.fit(X_train, y_train)
                col += 1
                
    def predict(self, X):
        predicts = np.empty((X.shape[0], 1))
        for fit in self.fits.values():
            predicts = np.concatenate([predicts, fit.predict(X).reshape(X.shape[0], 1)], axis=1)
            
        predicts=predicts[:,1:]
        predicts=np.mean(predicts,axis=1)
        return predicts

In [17]:
models = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"DecisionTree":DecisionTreeRegressor(random_state=0)
        }
bagging1 = bagging(3, models)
bagging1.fit(X_train, y_train)
bagging1.predict(X_test)

array([265090.77540816, 157208.43119596, 129819.2628537 , 237454.44517219,
       140504.44059613,  71151.72984213, 213720.72541143, 133288.94286901,
       496523.94552388, 175302.14122863, 201760.70178653, 203038.34228319,
       256194.24145011, 117658.93866877, 115955.63370914, 147830.98759866,
       243161.90819415, 154118.11323278, 148658.37102737, 154774.819012  ,
       149638.63475961, 150502.81891088, 105358.03048281, 202056.96070528,
       220125.0484722 , 112752.1189641 , 217188.72865822,  94372.12243891,
       236567.34518447, 128013.85138411, 193963.30575588, 229736.79873788,
       130104.34331564, 278210.68318023, 271569.25883688, 200689.51000115,
       221141.55400414, 121025.95325129, 271131.29807142, 313458.64088067,
       223060.80936334, 150272.42470238, 196635.1370681 , 246966.32461928,
       331276.77213968, 195426.15589589,  98037.63163153, 126297.13227631,
       205381.026414  ,  99065.31562028, 327173.86110444, 132500.03693136,
       170281.47763435,  

In [18]:
models_test_bag = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"LogisticRegression":LogisticRegression(random_state=0),
        #"DecisionTree":DecisionTreeRegressor(random_state=0),
        "bagging5":bagging(5, models),
        "bagging10":bagging(10, models),
        "bagging15":bagging(25, models),
        "bagging20":bagging(20, models)
        }

model_comparison(X_train, y_train, X_test, y_test, models_test_bag)

,LinearRegression,bagging5,bagging10,bagging15,bagging20
mean_squared_error,2.942067e+09,2.889374e+09,2.942423e+09,5.588843e+10,3.890309e+09
rmse,5.424082e+04,5.375289e+04,5.424411e+04,2.364073e+05,6.237234e+04


【問題3】スタッキングのスクラッチ実装

In [19]:
class stucking():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, split, n_iter, models):
        self.split = split
        self.models = models
        self.n_iter = n_iter

    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        
        n_block = round(X_train.shape[0]/self.split)
        if X_train.shape[0]%n_block is not 0:
            if X_train.shape[0]%n_block < n_block/2:
                n_block_0 = n_block + X_train.shape[0]%n_block
            else:
                n_block_0 = X_train.shape[0]%n_block
        else:
            n_block_0 = n_block
        
        self.X_train_b = X_train
        self.fits = [[] for i in range(3)]
        for stage in range(self.n_iter):            
            self.predicts_t = np.empty((X_train.shape[0], 1))  
            for model in self.models.values():
                i0=0
                self.predicts_k = 0  
                for k0 in range(self.split):
                    if k0==0:
                        i=n_block_0
                    else:
                        i=i0+n_block

                    X_test_k = self.X_train_b.iloc[i0:i]
                    y_test_k = y_train.iloc[i0:i]
                    X_train_k = self.X_train_b.drop(X_test_k.index, axis=0)
                    y_train_k = y_train.drop(y_test_k.index, axis=0)

                    fit_model = model.fit(X_train_k, y_train_k)
                    self.fits[2].append(stage)
                    self.fits[1].append(model)
                    self.fits[0].append(fit_model)
                    self.predicts_k = np.append(self.predicts_k, fit_model.predict(X_test_k))
                    i0=i
                    
                self.predicts_k = np.delete(self.predicts_k, 0)
                self.predicts_t = np.concatenate([self.predicts_t, self.predicts_k.reshape(X_train.shape[0], 1)], axis=1)
            
            self.predicts_t =np.delete(self.predicts_t, 0, 1)
            self.X_train_b = pd.DataFrame(self.predicts_t)
            
        self.last_model = list(self.models.values())[0].fit(self.X_train_b, y_train)

    def predict(self, X):
        self.X_blend_test = X
        self.predict_main = [[] for i in range(3)]
        for stage in range(self.n_iter):
            self.blend_test1 = np.empty((X_train.shape[0], 1))
            for model in range(len(list(self.models.values()))):
                self.blend_test_a = np.zeros((1,1))
                for k0 in range(self.split):
                    display("test00:{}".format(self.blend_test_a.shape))
                    self.blend_test_a = np.append(self.blend_test_a, self.fits[0][k0].predict(self.X_blend_test))
                    #display("test0_1{}".format(self.blend_test_a.shape))
                self.blend_test_a = np.delete(self.blend_test_a, 0)
                self.blend_test1 = np.concatenate([self.blend_test1, self.blend_test_a.reshape(self.blend_test_a.shape[0], 1)], axis=1)               

            self.blend_test1 =np.delete(self.blend_test1, 0, 1)
            self.X_blend_test = pd.DataFrame(self.blend_test1)
            
        self.last_predict = self.last_model.predict(self.X_blend_test)
        return self.last_predict


In [20]:

class stucking():
    """
    ブレンディングを行う。
    Parameters
    ----------
    kf_splits : int
      Trainﾃﾞｰﾀを何分割するか
    models : dict
      ブレンディングに使うモデル。
        ・keys:モデル名(str)
        ・value:モデルを適用するclass(class)
    """
    def __init__(self, split, n_iter, models):
        self.split = split
        self.models = models
        self.n_iter = n_iter

    def fit(self, X_train, y_train):
        """
        ブレンディングの学習を行う。
        Parameters
        ----------
        df_train : DataFrame
          学習データ。(targetﾃﾞｰﾀ含む)
        target_name : str
          学習データの目標値のFutureの名前。
        fits : dict
          ブレンディングの推測に使う学習モデル。
            ・keys :モデリング順につけた番号(int)
            ・value:学習モデル(class)
        n : int
          モデリング順につける番号。fitsのkeyにつかう番号
        Attributes
        ----------
        """
        df_train = pd.concat([X_train, y_train], axis=1)
        target_name = df_train.columns[-1]
        
        n_block = round(X_train.shape[0]/self.split)
        if X_train.shape[0]%n_block is not 0:
            if X_train.shape[0]%n_block < n_block/2:
                n_block_0 = n_block + X_train.shape[0]%n_block
            else:
                n_block_0 = X_train.shape[0]%n_block
        else:
            n_block_0 = n_block
        
        self.X_train_b = X_train
        self.fits = [[] for i in range(3)]
        for stage in range(self.n_iter):            
            self.predicts_t = np.empty((X_train.shape[0], 1))  
            for model in self.models.values():
                i0=0
                self.predicts_k = 0  
                for k0 in range(self.split):
                    if k0==0:
                        i=n_block_0
                    else:
                        i=i0+n_block

                    X_test_k = self.X_train_b.iloc[i0:i]
                    y_test_k = y_train.iloc[i0:i]
                    X_train_k = self.X_train_b.drop(X_test_k.index, axis=0)
                    y_train_k = y_train.drop(y_test_k.index, axis=0)

                    fit_model = model.fit(X_train_k, y_train_k)
                    self.fits[2].append(stage)
                    self.fits[1].append(model)
                    self.fits[0].append(fit_model)
                    self.predicts_k = np.append(self.predicts_k, fit_model.predict(X_test_k))
                    i0=i
                    
                self.predicts_k = np.delete(self.predicts_k, 0)
                self.predicts_t = np.concatenate([self.predicts_t, self.predicts_k.reshape(X_train.shape[0], 1)], axis=1)
            
            self.predicts_t =np.delete(self.predicts_t, 0, 1)
            self.X_train_b = pd.DataFrame(self.predicts_t)
            
        self.last_model = list(self.models.values())[0].fit(self.X_train_b, y_train)

    def predict(self, X):
        self.X_blend_test = X
        self.predict_main = [[] for i in range(3)]
        for stage in range(self.n_iter):
            self.blend_test1 = np.empty((self.X_blend_test.shape[0], 1))
            for model in range(len(list(self.models.values()))):
                blend = np.empty((self.X_blend_test.shape[0], 1))
                for k0 in range(self.split):
                    #display("brend:{}".format(blend.shape))
                    #display(self.fits[0][k0].predict(self.X_blend_test).shape)
                    blend = np.concatenate([blend, self.fits[0][k0].predict(self.X_blend_test).reshape(-1,1)], axis=1)
                blend = np.delete(blend, 0, 1)
                blend = np.mean(blend, axis=1)
                self.blend_test1 = np.concatenate([self.blend_test1, blend.reshape(blend.shape[0], 1)], axis=1)               

            self.blend_test1 =np.delete(self.blend_test1, 0, 1)
            self.X_blend_test = pd.DataFrame(self.blend_test1)
        self.last_predict = self.last_model.predict(self.X_blend_test)
        return self.last_predict


In [21]:
models = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        "DecisionTree":DecisionTreeRegressor(random_state=0)
        }

stucking_0 = stucking(5, 3, models)
stucking_0.fit(X_train, y_train)
#stucking_0.fits
stucking_0.predicts_k.shape

(1168,)

In [22]:
stucking_0.predict(X_test)

array([15358.18275311, 14506.13408846, 14165.39243358, 14910.35875767,
       14064.84202106, 14000.18022182, 14403.61516107, 14321.52970407,
       17141.48535632, 14074.4888415 , 14301.78481269, 14599.44713542,
       14875.27673285, 13890.23717586, 14055.36378556, 14266.4148839 ,
       14820.6157824 , 13972.50676668, 14188.38839489, 14731.13992381,
       14330.87307163, 13843.59039122, 13813.80962448, 14418.84983394,
       14777.58702573, 14654.22347614, 14502.58349445, 13644.8445168 ,
       14616.332631  , 14177.15949532, 14686.41238817, 14686.0933864 ,
       13901.33120749, 15070.73912996, 14932.69854735, 14387.06683534,
       14491.75919871, 13873.95858602, 14976.24860769, 15456.07476214,
       15033.53424465, 14308.19742169, 14263.58789549, 14741.73081985,
       15612.44805153, 14642.87918631, 13821.85619769, 13884.9346082 ,
       14602.20799946, 13917.15461802, 15498.04274335, 14047.46893879,
       14284.93303409, 13787.98536516, 14566.94053122, 13864.85123736,
      

In [23]:
models_test_bag = {
        "LinearRegression" : LinearRegression(),
        #"SVC":SVC(gamma="auto"),
        #"LogisticRegression":LogisticRegression(random_state=0),
        #"DecisionTree":DecisionTreeRegressor(random_state=0),
        "stucking5":stucking(5, 3, models),
        "stucking10":stucking(10, 3, models),
        "stucking15":stucking(15, 3, models),
        }

model_comparison(X_train, y_train, X_test, y_test, models_test_bag)

,LinearRegression,stucking5,stucking10,stucking15
mean_squared_error,2.942067e+09,3.471547e+10,3.735432e+10,3.639037e+10
rmse,5.424082e+04,1.863209e+05,1.932727e+05,1.907626e+05
